In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
from arc import*
#from qutip import *
from time import time

from basics import *
#from hybrid_matrix_elements import *
#from atomic_physics import *

pi = np.pi

In [ ]:
class RydStateFS:
    """
    Tiny wrapper class to hold quantum numbers of rydberg state in FS basis
    """
    def __init__(self, n, l, j,**kwargs):
        """
        Args:
            n : principle quantum number (int)
            l : orbital angular momentum quantum number (int)
            j : FS angular momentum quantum number (int/half int)
        kwargs:
            for additional desired quantum numbers
        """
        self.n = n
        self.l = l
        self.j = j
        self.__qn = {"n": n, "l": l, "j": j}
        self.__qn.update(kwargs)
        if "atom" in kwargs.keys():
            self.atom_name = kwargs["atom"]
        else:
            self.atom_name = "Cesium"
        self.atom = eval(f"{self.atom_name}()")
    def __getitem__(self,arg):
        return self.__qn[arg]
    def quantum_numbers(self):
        return self.__qn
    def get_energy(self):
        return self.atom.getEnergy(self.n,self.l,self.j)
    def __repr__(self):
        return f"RydStateFS({','.join([f'{key}={value}' for key,value in self.__qn.items()])})"
    def __str__(self):
        return self.__repr__()
    # Comparison operators check energy levels of compared states
    def __lt__(self, other):
        return self.get_energy() < other.get_energy()
    def __eq__(self, other):
        return self.get_energy() == other.get_energy()
    def ket(self):
        L_labels = {0: "S", 1: "P", 2: "D",3:"F"}
        if list(self.__qn.keys()) == ["n","l","j"]:
            return f"|{self.n},{L_labels[self.l]},{int(2*self.j)}/2>"
        else:
            extra_qn = {q_l: q_v for q_l, q_v in self.__qn.items() if q_l not in ["n","l","j"]}
            for q_l, q_v in extra_qn.items():
                if type(q_v) is not int:
                    extra_qn[q_l] = f"{int(q_v*2)}/2"
            return f"|{self.n},{L_labels[self.l]},{int(2*self.j)}/2;" + ",".join([
                f"{q_l}={q_v}" for q_l, q_v in extra_qn.items()
            ]) + ">"
    
def detuning(state1: RydStateFS, state2: RydStateFS, field_nu: float):
    """
    Returns detuning of field from bare resonance between state1 and state2
    Args:
        state1: Rydberg state in FS basis for the first state
        state2: Rydberg state in FS basis for the second state
        field_nu: radial frequency of a single tone field coupling state1 and state2 (Hz)
    Returns:
        detuning : field detuning from bare resonance in Hz (radial frequency)
    """
    bare_res = (state1.get_energy()-state2.get_energy())*e/hb
    return bare_res - field_nu
def states_sgn(state1: RydStateFS, state2: RydStateFS):
    return -np.sign(state1.get_energy()-state2.get_energy())
tau = 2*pi

In [ ]:
def matrix_print(mat,mult):
    print("\n".join([
        "[" + ",".join([f"{me.real*mult:7.3f}" for me in row]) + "]"
    for row in mat]))

In [ ]:
def get_energy_nstate(state):
    return state.get_energy()*e/hb+state["nphot"]*field_omega

def build_H0(basis: list,t_state: RydStateFS=None)->np.ndarray:
    """
    Build Bare atomic hamiltonian
    
    State energies computed using arc. Computed for states in cesium.
    Args:
        basis: list of states making up the basis. Each item should be a RydStateFS object
        t_state: "target" state, this state is the state we care about. This state defines 0 energy 
            (all energies measured against it). If None, energies reported are relative to ionization energy
    Returns:
        H0: Diagonal hamiltonian of the unperturbed cesium atom. Diagonal entries are unperturbed energies of basis states. 
            energies are reported relative to the energy of t_state (or ionization energy), energies reported in frequency 
            (radial), units=Hz
    """
    if t_level is not None:
        H0 = np.diag(
            np.array(
                [detuning(state,t_state,0) for state in basis],
                dtype=complex
            )
        )
    else:
        H0 = np.diag(
            np.array(
                [state.get_energy()*e/hb for state in basis],
                dtype=complex
            )
        )
    return H0

def dipoles(basis: list, qs=[1,0,0]):
    """
    Builds the matrix of dipole moments between basis states. Assumes dipole field is pi-polarized.
    Dipole moments computed using arc. Computed for rydberg states in cesium.
    
    Args:
        basis: list of states that make up our basis
        qs: list of proportions of polarization states in driving field. For dc fields should always be [1,0,0] (rotations should
            be accounted for later). if qs is not normalized it's normalize within the function. indeces are [pi,sigma_plus,sigma_minus]
    Returns:
        d : Matrix of dipole moments (divided by hbar) between basis states
    """
    qs = np.array(qs)/sum([abs(q) for q in qs])  # normalize qs
    
    d = np.zeros((len(basis),len(basis)), dtype=complex)
    for q in [-1,0,1]:
        for i, state in enumerate(basis):
            for j, statep in enumerate(basis):
                if abs(statep.l-state.l) != 1:
                    continue
                if abs(statep.j-state.j) > 1:
                    continue
                if statep.get_energy() > state.get_energy():
                    continue
                    print(f"Computing Matrix Element {state.ket()} x {statep.ket()}")
                d[i,j] += qs[q]*state.atom.getDipoleMatrixElement(
                    statep.n,
                    statep.l,
                    statep.j,
                    statep["mj"],
                    state.n,
                    state.l,
                    state.j,
                    state["mj"],
                    q
                )*ao*e/hb
    return d + np.conjugate(d.T)

def build_floquet(Hdc, d, Eac, nmax, omega_d):
    """
    Build Floquet hamiltonian.
    Args:
        Hdc: Hamiltonian describing time-independent system. Ostensibly a bare atomic Hamiltonian + DC electric fields
            Matrix elements should be reported in Radial Frequency (Hz)
        d: Matrix of dipole moments between states in the basis. Matrix elements should be reported in 
            Radial Frequency/Electric Field Strength (Hz/(V/m))
        Ea: Electric field strength of AC field (V/m)
        omega_d: radial frequency of driving field. (Hz)
    Returns:
        Floquet Hamiltonian constructed in the Shirley-Floquet formalism. Matrix elements reported in radial frequency (Hz).
    """
    def Hdc_n(Ham, n, omega_d):
        '''Build Diagonal block for Floquet hamiltonian'''
        return Ham+np.diag(n*omega_d*np.ones(Ham.shape[0]))
    
    Hfloquet = np.zeros([dim*(2*nmax+1) for dim in Hdc.shape],dtype=complex)
    for i,ni in enumerate(range(-nmax,nmax+1)):
        for j,nj in enumerate(range(-nmax,nmax+1)):
            if i == j:
                block = Hdc_n(Hdc, ni, omega_d)
            elif abs(i-j) == 1:
                block = Eac*d/2
            else:
                continue
            Hfloquet[len(basis)*i:len(basis)*(i+1),len(basis)*j:len(basis)*(j+1)] = block
    return Hfloquet

In [ ]:
# Build a Basis
s, p, d, f = [0,1,2,3]
t_level = RydStateFS(52,p,3/2)

max_det = tau*100e9  # GHz
levels = [] # [t_level,RydStateFS(51,d,3/2)]
# Positive
m = True
p = True
dn = 0
while (m or p):
    n_p = t_level.n+dn
    dn += 1
    # print(f"np = {n_p}")
    for lp in range(t_level.l-1,t_level.l+2,1):
        # print(f"lp = {lp}")
        if lp < 0 or abs(lp - t_level.l) > 1:
            # print(f"lp = {lp}, t.l = {t_level.l}, abs(t_level.l-lp) = {abs(t_level.l-lp)}")
            continue
        if (lp < t_level.l and m) or (lp > t_level.l and p) or lp == t_level.l:
            new_levels = [RydStateFS(n_p,lp,lp+1/2),RydStateFS(n_p,lp,lp-1/2)]
            # print([level.ket() + f"{abs(detuning(t_level,level, 0))*1e-9/tau}" for level in new_levels])
            if any([abs(detuning(t_level,new_level,0)) < max_det for new_level in new_levels]):
                levels.extend(new_levels)
            else:
                # print("detuning test failed")
                if lp == t_level.l:
                    continue
                # print(m,p)
                m = m and not (lp < t_level.l)
                p = p and not (lp > t_level.l)
                # print(m, p)
        # else:
            # print("bad L")
            # print(t_level.l, lp, p, lp>t_level.l and p)
            # print(lp < t_level.l and m, lp > t_level.l and p)

# Negative
m = True
p = True
dn = -1
while (m or p):
    n_p = t_level.n+dn
    dn -= 1
    # print(f"np = {n_p}")
    for lp in range(t_level.l-1,t_level.l+2,1):
        # print(f"lp = {lp}")
        if lp < 0 or abs(lp - t_level.l) > 1:
            # print(f"lp = {lp}, t.l = {t_level.l}, abs(t_level.l-lp) = {abs(t_level.l-lp)}")
            continue
        if (lp < t_level.l and m) or (lp > t_level.l and p) or lp == t_level.l:
            new_levels = [RydStateFS(n_p,lp,lp+1/2),RydStateFS(n_p,lp,lp-1/2)]
            # print([level.ket() + f"{abs(detuning(t_level,level, 0))*1e-9/tau}" for level in new_levels])
            if any([abs(detuning(t_level,new_level,0)) < max_det for new_level in new_levels]):
                levels.extend(new_levels)
            else:
                # print("detuning test failed")
                if lp == t_level.l:
                    continue
                # print(m,p)
                m = m and not (lp < t_level.l)
                p = p and not (lp > t_level.l)
                # print(m, p)
        # else:
            # print("bad L")
            # print(t_level.l, lp, p, lp>t_level.l and p)
            # print(lp < t_level.l and m, lp > t_level.l and p)
print("[\n\t" + ",\n\t".join([level.ket() for level in levels]) + "]")

basis = []
for level in levels:
    if level.j < 0:
        continue
    basis.extend([RydStateFS(level.n,level.l,level.j,mj=m) for m in np.arange(-level.j*0+1/2,level.j+1)])
basis.sort()
        
print("[\n\t" + ",\n\t".join([s.ket() for s in basis]) + "\n]")

In [ ]:
print(len(basis))

In [ ]:
# Experiment Parameters
Edc = 0  # V/m
Eac = 80  #V/m
n_p =  6 #  Number of floquet photons to consider
ns = range(-n_p,n_p+1)
field_omega = tau*4.780*1e9

ac_polarization = [1,0,0]

# Eac_list = np.linspace(0,60,100)  # V/m
Edc_list = np.linspace(0,20,100)  #V/m
basis_n = []
for n in ns:
    basis_n.extend([eval(state.__str__()[:-1]+f",nphot={n})") for state in basis])

In [ ]:
def inner(v1,v2):
    return sum([conjugate(c1)*c2 for c1, c2 in zip(v1, v2)])

# Build a floquet Hamiltonian
H0 = build_H0(basis, t_state=t_level)

d_mat = dipoles(basis,ac_polarization)

#Energies = np.ones((len(basis)*(2*n_p+1),len(Eac_list)), dtype=float)
#EVectors = np.zeros((len(basis)*(2*n_p+1),len(basis)*(2*n_p+1),len(Eac_list)), dtype=complex)
Energies = np.ones((len(basis)*(2*n_p+1),len(Edc_list)), dtype=float)
EVectors = np.zeros((len(basis)*(2*n_p+1),len(basis)*(2*n_p+1),len(Edc_list)), dtype=complex)


for i, Edc in enumerate(Edc_list):
#for i, Eac in enumerate(Eac_list):
    Hdc = H0 + Edc*d_mat
    Hfloquet = build_floquet(Hdc, d_mat, Eac, n_p, field_omega)
    
    t_start = time()
    print(f"Begin Diagonalization")
    eigenvalues, eigen_vectors = np.linalg.eig(Hfloquet)
#    print(f"Diagonalized Hfloquet for with dims {len(basis_n)}x{len(basis_n)} for index, field strength = {i}, {Eac:.2f}V/m")
    print(f"Diagonalized Hfloquet for with dims {len(basis_n)}x{len(basis_n)} for index, field strength = {i}, {Edc:.2f}V/m")
    print(f"Diagonalization Time: {time()-t_start}s" )
    eigen_vectors = eigen_vectors.T
    for j, ev in enumerate(eigen_vectors):
        ev = ev/(np.absolute(ev)**2).sum()
        eigen_vectors[j] = ev
        
    if i ==0:
        key=[0]*len(basis_n)
        for k, p0 in enumerate(eigen_vectors):
            j = np.where(p0 == max(p0))[0][0]
            #print(j)
            key[j] = int(k)
        EVectors[:,:,i] = eigen_vectors[key,:]
        Energies[:,i] = np.real(eigenvalues[key])
    else:
        # Compute all inner products at once with matrix multiplication. Find indeces that 
        ips = np.absolute(np.dot(np.conjugate(eigen_vectors),EVectors[...,i-1].T))**2
        inds = np.argwhere(ips>0.5)
        # print(ips.shape,ips)
        if any([a not in inds[:,1] or a not in inds[:,0] for a in range(len(basis_n))]):
            raise RuntimeError(
                f"Failed to represent all basis states with inds: {inds}. \n"+
                f"Inner product matrix = {matrix_print(ips,1)}")
        for ind in inds:
            EVectors[ind[1],:,i] = eigen_vectors[ind[0],:]
            Energies[ind[1],i] = real(eigenvalues[ind[0]])
    print(f"Finish Rearangement after time {time()-t_start}")

In [ ]:
thrs = 5e-2  # threshold for caring about population
for i, state in enumerate(basis):
    if [state.n,state.l,state.j] == [t_level.n, t_level.l, t_level.j]:
        interesting_inds = []
        #print(state.ket())
        ii = i+len(basis)*n_p
        #print(ii)
        for k, p in enumerate(EVectors[:,ii,:]):
            #check 
            #print(k)
            if max(np.absolute(p)**2)>thrs:
                interesting_inds.append(k)
                #print(f"index {k} is interesting")
                #print(f"corresponds to {basis_n[k].ket()}")
                #print(f"{p}")
                #print(max(p))
            #print(k,p.shape,p[1:].max())
        fig,axar = plt.subplots(1,2,figsize=(8,6))
        for k in interesting_inds:
#            axar[0].plot(Eac_list, 1e-6*np.real(Energies[k,:])/tau, label=basis_n[k].ket())
            axar[0].plot(Edc_list, 1e-6*np.real(Energies[k,:])/tau, label=basis_n[k].ket())
#            axar[1].plot(Eac_list[:], np.absolute(EVectors[k,ii,:])**2, label=basis_n[k].ket())
            axar[1].plot(Edc_list[:], np.absolute(EVectors[k,ii,:])**2, label=basis_n[k].ket())
        axar[0].set_title("Energies")
        #axar[0].set_ylim(-1e3,6e3)
        axar[1].set_title("Probability")
        axar[1].set_yscale("log")
        axar[1].set_ylim(thrs*0.8,1)
        axar[0].legend()
        axar[0].set_ylabel("Rydberg State Energy/$2\pi$ (MHz)")
#        axar[0].set_xlabel("AC Electric Field strength shift/$2\pi$ (MHz)")
        axar[0].set_xlabel("DC Electric Field strength shift/$2\pi$ (MHz)")

        #axar[1].legend()
        fig.suptitle(f"State = {state.ket()}")
        fig.tight_layout()
        fig.show()

        

In [ ]:
thrs = 5e-1  # threshold for caring about population
fig,axar = plt.subplots(1,2,figsize=(8,6))
for i, state in enumerate(basis):
    if [state.n,state.l,state.j] == [t_level.n, t_level.l, t_level.j]:
        interesting_inds = []
        #print(state.ket())
        ii = i+len(basis)*n_p
        #print(ii)
        for k, p in enumerate(EVectors[:,ii,:]):
            #check 
            #print(k)
            if max(np.absolute(p)**2)>thrs:
                interesting_inds.append(k)
                #print(f"index {k} is interesting")
                #print(f"corresponds to {basis_n[k].ket()}")
                #print(f"{p}")
                #print(max(p))
            #print(k,p.shape,p[1:].max())
        for k in interesting_inds:
#            axar[0].plot(Eac_list, 1e-6*np.real(Energies[k,:])/tau, label=basis_n[k].ket())
#            axar[1].plot(Eac_list[:], np.absolute(EVectors[k,ii,:])**2, label=basis_n[k].ket())
            axar[0].plot(Edc_list, 1e-6*np.real(Energies[k,:])/tau, label=basis_n[k].ket())
            axar[1].plot(Edc_list[:], np.absolute(EVectors[k,ii,:])**2, label=basis_n[k].ket())
axar[0].set_title("Energies")
#axar[0].set_ylim(-1e3,6e3)
axar[1].set_title(f"Overlap with {t_level.ket()}")
axar[1].set_yscale("log")
#axar[1].set_ylim(thrs*0.8,1)
axar[0].legend()
axar[0].set_ylabel("Rydberg State Energy/$2\pi$ (MHz)")
#axar[0].set_xlabel("AC Electric Field strength (V/m))")
axar[0].set_xlabel("DC Electric Field strength (V/m))")

        #axar[1].legend()
#fig.suptitle(f"State = {state.ket()}")
fig.tight_layout()
fig.show()


In [ ]:
thrs = 0.5  # threshold for caring about population
fig,axar = plt.subplots(1,2,figsize=(8,6))
for i, state in enumerate(basis):
    if [state.n,state.l,state.j] == [t_level.n, t_level.l, t_level.j]:
        interesting_inds = []
        #print(state.ket())
        ii = i+len(basis)*n_p
        #print(ii)
        for k, p in enumerate(EVectors[:,ii,:]):
            #check 
            #print(k)
            if max(np.absolute(p)**2)>thrs:
                interesting_inds.append(k)
                #print(f"index {k} is interesting")
                #print(f"corresponds to {basis_n[k].ket()}")
                #print(f"{p}")
                #print(max(p))
            #print(k,p.shape,p[1:].max())
        for k in interesting_inds:
#            axar[0].plot(Eac_list, 1e-6*np.real(Energies[k,:])/tau, label=basis_n[k].ket())
#            axar[1].plot(Eac_list[:], np.absolute(EVectors[k,ii,:])**2, label=basis_n[k].ket())
            axar[0].plot(Edc_list, 1e-6*np.real(Energies[k,:]-Energies[k,0])/tau, label=basis_n[k].ket())
            axar[1].plot(Edc_list[:], np.absolute(EVectors[k,ii,:])**2, label=basis_n[k].ket())
axar[0].set_title("Energies")
#axar[0].set_ylim(-1e3,6e3)
axar[1].set_title(f"Overlap with {t_level.ket()}")
axar[1].set_yscale("log")
axar[1].set_ylim(thrs*0.8,1)
axar[0].legend()
axar[0].set_ylabel("DC Stark Shift/$2\pi$ (MHz)")
#axar[0].set_xlabel("AC Electric Field strength (V/m))")
axar[0].set_xlabel("DC Electric Field strength (V/m))")

        #axar[1].legend()
fig.suptitle(f"State = {t_level.ket()}")
fig.tight_layout()
fig.show()

In [ ]:
E_field = 80  # V/m
O = E_field*t_level.atom.getDipoleMatrixElement(
    52,
    1,
    3/2,
    3/2,
    51,
    2,
    5/2,
    3/2,
    0
)*e*ao/hb
print(f"Rabi = 2pi x {O*1e-6/tau:.0f}MHz")

In [ ]:
def inner(v1,v2):
    return sum([conjugate(c1)*c2 for c1, c2 in zip(v1, v2)])

# Build a floquet Hamiltonian
H0 = build_H0(basis, t_state=t_level)

d_mat = dipoles(basis,ac_polarization)

Energies = np.ones((len(basis)*(2*n_p+1),len(Eac_list)), dtype=float)
EVectors = np.zeros((len(basis)*(2*n_p+1),len(basis)*(2*n_p+1),len(Eac_list)), dtype=complex)

for i, Eac in enumerate(Eac_list):
    Hdc = H0 + Edc*d_mat
    Hfloquet = build_floquet(Hdc, d_mat, Eac, n_p, field_omega)
    
    t_start = time()
    print(f"Begin Diagonalization")
    eigenvalues, eigen_vectors = np.linalg.eig(Hfloquet)
    print(f"Diagonalized Hfloquet for with dims {len(basis_n)}x{len(basis_n)} for index, field strength = {i}, {Eac:.2f}V/m")
    print(f"Diagonalization Time: {time()-t_start}s" )
    eigen_vectors = eigen_vectors.T
    for j, ev in enumerate(eigen_vectors):
        ev = ev/(np.absolute(ev)**2).sum()
        eigen_vectors[j] = ev
        
    if i ==0:
        key=[0]*len(basis_n)
        for k, p0 in enumerate(eigen_vectors):
            j = np.where(p0 == max(p0))[0][0]
            #print(j)
            key[k] = int(j)
        EVectors[:,:,i] = eigen_vectors[key,:]
        Energies[:,i] = np.real(eigenvalues[key])
    else:
        for eigen_value, eigen_vector in zip(eigenvalues,eigen_vectors):
            broken = False
            for j, old_vec in enumerate(EVectors[...,i-1]):
                if np.absolute(np.dot(conjugate(eigen_vector),old_vec))**2 > 0.5:
                    EVectors[j,:,i] = eigen_vector
                    Energies[j,i] = real(eigen_value)
                    print(f"Loop broken successfully, ind = {j}")
                    broken = True
                    break
            if not broken:
                raise RunTimeError(f"Failed to write eigenvalue, vector pair {eigen_value, eigen_vector}")
    print(f"Finish Rearangement after time {time()-t_start}")

In [ ]:
fig,ax = plt.subplots(1,1)
for Es in E_sorted:
    ax.scatter(Eac_list,Es*1e-6/tau)
ax.set_ylim(-2e2,2e2)
fig.show()

In [ ]:
fig,ax = plt.subplots(1,1)
ax.scatter(Eac_list,E_sorted[1,:]*1e-6/tau)
ax.set_ylim(-2e2,2e2)
fig.show()

In [ ]:
fig,ax = plt.subplots(1,1)
for i in range(len(basis_n)):
    ax.scatter(Eac_list,np.real(P_sorted[0,i,:]))
#ax.set_ylim(-2e2,2e2)
ax.set_yscale("log")
fig.show()

In [ ]:
def get_energy_nstate(state):
    return state.get_energy()*e/hb+state["nphot"]*field_omega

energy_threshold = tau*1e6*1000  # MHz
#thrs = 1e-1  # threshold for caring about population
for t_s in [eval(f"{t_level.__str__()[:-1]}, mj={m}, nphot={0})") for m in np.arange(1/2,t_level.j+1)]:
    fig, axar = plt.subplots(1,2,figsize=(8,6))
    for k, state in enumerate(basis_n):
        if state.ket() == t_s.ket():
            print(f"t_s = {t_s.ket()}\nstate = {state.ket()}")
            ii = k
            break
    for i, state in enumerate(basis_e):
        if any(np.absolute(Energies[i,:]) < energy_threshold):
            axar[0].plot(Eac_list,np.real(Energies[i,:]*1e-6/tau),label=state.ket())
            axar[1].plot(Eac_list,Probs[i,ii,:],label=state.ket())
    axar[0].legend()
    axar[1].legend()
    fig.suptitle(basis_n[ii].ket())
    fig.show()

In [ ]:
j=0
for n in range(-n_p,n_p+1):
    for i, state in enumerate(basis):
        print(j,state.ket(),n)
        j+=1

In [ ]:
print("[\n\t"+",\n\t".join([state.ket() for state in basis_n]) + "]")

In [ ]:
e_val, evec = np.linalg.eig(Hfloquet)
matrix_print(evec.T,1)
evec=evec.T
matrix_print(evec,1)
print((np.absolute(evec[0])**2).sum())
for i, vec in enumerate(evec):
    vec = vec/(np.absolute(vec)**2).sum()
    evec[i] = vec
print()
matrix_print(evec,1)
print(evec[0])

In [ ]:
len(basis)

In [ ]:
print(e_val[np.argsort(e_val)])
print(evec[np.argsort(e_val)])

In [ ]:
# Build Hamiltonian in RWA

#Bare atomic Hamiltonian
H0_rwa = np.diag(
    np.array(
        [detuning(state,t_level,field_omega) for state in basis],
        dtype=complex
    )
)

i_t_state = [i for i, state in enumerate(basis) if [state.n,state.l,state.j] == [t_level.n, t_level.l, t_level.j]]
for i in i_t_state:
    H0_rwa[i,i] = 0

#Electric field strength of driving field (V/m)
Energies_rwa = 1j*np.ones((len(basis),len(Eac_list)), dtype=complex)
for i,Eac in enumerate(Eac_list):
    Hrwa = H0_rwa + Eac*d_mat/2
    eigenvalues, eigen_vectors = np.linalg.eig(Hrwa)
    # normailze eigen_vectors for ease of debugging
    for j, ev in enumerate(eigen_vectors.T):
        new_v = ev/sum(ev**2)
        eigen_vectors[:,j] = new_v
    #assign Energies to states in basis that have the greatest overlap with a given eigenvector
    probs = np.absolute(eigen_vectors)**2
    for eig, p in zip(eigenvalues, probs.T):
        ind = np.where(p==max(p))[0]
        Energies_rwa[ind,i] = eig
        
Energies_rwa[Energies_rwa == 1j] = np.NaN

In [ ]:
n0_energies = Energies[n_p*len(basis):(n_p+1)*len(basis),:]
Energies[Energies==1j] = np.NaN

fig,axar = plt.subplots(len(levels),2,figsize=(10,5*len(levels)))
for i, state in enumerate(basis):
    for j, level in enumerate(levels):
        if [state.n,state.l,state.j] == [level.n, level.l, level.j]:
            for k, n in enumerate(range(-n_p,n_p+1)):
                n_energies = Energies[k*len(basis):(k+1)*len(basis),:]
                labeled = True
                axar[j,0].plot(Eac_list,1e-6*(np.real(n_energies[i,:]-Hfloquet[i+k*len(basis),i+k*len(basis)]))/tau, label = f"{state.ket()}; n={n}")
                # axar[j,0].legend()
                axar[j,0].set_title(f"Effective Stark shifts for {level.ket()}. Shirley Floquet")
                axar[j,0].set_xlabel(f"AC Electric Field Strength (V/m)")
                axar[j,0].set_ylabel(f"AC stark shift (MHz)")
            if state["mj"] == 3/2:
                axar[j,1].plot(Eac_list,np.real(1e-6*(n0_energies[i,:] - n0_energies[i-1,:])/tau))
                axar[j,1].legend()
                axar[j,1].set_title(f"1/2-3/2 splitting for {level.ket()}")
                axar[j,1].set_xlabel(f"AC Electric Field Strength (V/m)")
                axar[j,1].set_ylabel(f"Tensor shift (MHz)")
fig.tight_layout()
fig.show()

interesting_levels = [
    RydStateFS(52,1,3/2),
    RydStateFS(51,2,3/2),
    RydStateFS(51,2,5/2)
]

fig,ax = plt.subplots(1,1,figsize=(10,10))
for i, state in enumerate(basis):
    for level in interesting_levels:
        if [state.n,state.l,state.j] == [level.n, level.l, level.j]:
            ax.plot(Eac_list,np.real(1e-6*(n0_energies[i,:])/tau),label = level.ket())
ax.legend()
fig.tight_layout()
fig.show()

In [ ]:
n_energies[i,:]-n*H0[i,i]

In [ ]:
matrix_print(d_mat,1e-9)

In [ ]:
matrix_print(d_mat,1e-9)

In [ ]:
Hfloquet.shape

In [ ]:
dir(axar[2,0])

In [ ]:
axar[2,0].get_legend

In [ ]:
s, p, d, f = [0,1,2,3]
t_level = RydStateFS(52,p,3/2)

max_det = tau*20e9  # GHz
# Positive
m = True
p = True
while m and p:
    dn += 1
    n_p = t_level.n+1
    for lp in range(t_level.l-1,t_level.l+2,2):
        if lp < 0:
            continue
        if (lp < t_level.l and m) or (lp > t_level.l and p):
            new_levels = [RydStateFS(n_p,lp,lp+1/2),RydStateFS(n_p,lp,lp-1/2)]
            if any([detuning(t_state,new_level,0) < max_det for new_level in new_levels]):
                levels.extend(new_levels)
            else:
                m = lp < t_level.l
                p = lp > t_level.l
                
ad_levels = [
    RydStateFS(52,p,1/2),
    RydStateFS(52,p,3/2),
    RydStateFS(51,d,3/2),
    RydStateFS(51,d,5/2)
]
levels = ad_levels

basis = []
for level in levels:
    for m in np.arange(-level.j,level.j+1):
        basis.append(RydStateFS(level.n,level.l,level.j,mj=m))

print("[\n\t" + ",\n\t".join([s.ket() for s in basis]) + "\n]")

In [ ]:
# Build Hamiltonian in RWA

field_nu = 4.780*1e9*tau

#Bare atomic Hamiltonian
H0 = np.diag(
    np.array(
        [detuning(state,t_level,field_nu) for state in basis],
        dtype=complex
    )
)

for i in range(2,6):
    H0[i,i] = 0

#Dipole matrix elements
d = np.zeros(H0.shape, dtype=complex)
for i, state in enumerate(basis):
    for j, statep in enumerate(basis):
        if abs(statep.l-state.l) != 1:
            continue
        if abs(statep.j-state.j) > 1:
            continue
        d[j,i] = state.atom.getDipoleMatrixElement(
            statep.n,
            statep.l,
            statep.j,
            statep["mj"],
            state.n,
            state.l,
            state.j,
            state["mj"],
            0
        )*ao*e/hb

#Electric field strength of driving field (V/m)
Es = np.linspace(0,100,100)
Energies = np.zeros((len(basis),len(Es)))
for i,E in enumerate(Es):
    H = H0 + E*d/2
    Energies[:,i], Eigenvectors = np.linalg.eig(H)
    Energies[:,i].sort()

In [ ]:
fig,axar = plt.subplots(len(levels),1,figsize=(8,6*len(levels)))
for i, state in enumerate(basis):
    for j, level in enumerate(levels):
        if [state.n,state.l,state.j] == [level.n, level.l, level.j]:
            axar[j].plot(Es,1e-6*(Energies[i,:]-real(H[i,i]))/tau,label = state.ket())
            #axar[j].legend()
            axar[j].set_title(f"AC stark shifts on {level.ket()}")
            axar[j].set_xlabel(f"Electric Field Strength (V/m)")
            axar[j].set_ylabel(f"AC Stark Shift (MHz)")
fig.tight_layout()
fig.show()